In [55]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [56]:
import h5py
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten, Dropout
from keras.layers import concatenate
from keras.layers.convolutional import Conv2D
from keras.optimizers import adadelta,RMSprop,SGD,Adam
from keras.layers.convolutional import MaxPooling2D, AveragePooling2D
import tensorflow as tf

In [57]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

TRAIN_DATASET_PATH = 'data/uci_ml_hackathon_fire_dataset_2012-05-09_2013-01-01_10k_train.hdf5'
TEST_DATASET_PATH = 'data/uci_ml_hackathon_fire_dataset_2013-01-01_2014-01-01_5k_test.hdf5'

def getDataDict(DatasetPath):
    with h5py.File(DatasetPath, 'r') as f:
        data = {}
        for k in list(f):
            data[k] = f[k][:]
        return data

train_data = getDataDict(TRAIN_DATASET_PATH)

train_data.keys()

def transformDateTime(datetime):
    ret = np.asarray([x * np.ones((1,30, 30)) for x in datetime])
    return ret


def transformLandCover(landCover):
    nanConvert = {
        0: 0,
        1: 0,
        2: -1,
        3: 0,
        4: -1,
        5: 0,
        6: 0,
        16:0
    }
    ret=[]
    
    for datapoint in landCover:
        for i in range(17):
            if i in nanConvert.keys():
                datapoint[i][np.isnan(datapoint[i])]= nanConvert[i] 
        ret.append(datapoint)
    return np.asarray(ret)


def transformLatAndLong(val):
    ret = np.asarray([x * np.ones((1,30, 30)) for x in val])
    return ret

#TODO : define temperature according to datetime average
def transformMet(met):
    nanConvert = {
        0: 290,
        1: 26,
        2: 0,
        3: 0,
        4: 0,
    }
    met0=[]
    met1=[]
    for datapoint in met:
        for i in range(5):
            datapoint[0][i][np.isnan(datapoint[0][i])]= nanConvert[i] 
            datapoint[1][i][np.isnan(datapoint[1][i])]= nanConvert[i]
            if(i==2):
                datapoint[0][i]= sigmoid(datapoint[0][i]- 0.4232)
                datapoint[1][i]= sigmoid(datapoint[1][i]- 1.4365)
            elif(i==3):
                datapoint[0][i]= sigmoid(datapoint[0][i]+0.0854)
                datapoint[1][i]= sigmoid(datapoint[1][i]- 0.495)
        met0.append(datapoint[0])
        met1.append(datapoint[1])
    return np.asarray(met0),np.asarray(met1)

def transformFire(fire):
    return np.asarray(fire)

#transform all of them into dict of 3d np arrays.
#Augmentation step must take place after this.
#Can store this in h5py file after this.
def transformAndClean(data):
    X = {}
    Y= {}
    X['datetime'] = transformDateTime(data['datetime'])
    X['landCover'] = transformLandCover(data['land_cover'])
    X['latitude'] = transformLatAndLong(data['latitude'])
    X['longitude'] = transformLatAndLong(data['longitude'])
    X['met0'], X['met1'] = transformMet(data['meteorology'])
    X['observed'] = transformFire(data['observed'])
    Y['target'] = transformFire(data['target'])
    return X,Y

X,Y = transformAndClean(train_data)
train_data=None

startDictionary={
    'datetime':0,
    'landCover':1,
    'latitude':18,
    'longitude':19,
    'met0':20,
    'met1':25,
    'observed':30,
    'target':0
}

lengthDictionary={
    'datetime':1,
    'landCover':17,
    'latitude':1,
    'longitude':1,
    'met0':5,
    'met1':5,
    'observed':5,
    'target':2
}
def flattenData(data):
    length =0
    for key,value in data.items():
        
        length += value.shape[1]
        n=value.shape[0]
    ret = np.zeros((n,length,30,30))
    for key,arr in data.items():    
        for index,datapoint in enumerate(arr):
            ret[index][startDictionary[key]: startDictionary[key]+lengthDictionary[key]][:][:]=datapoint        
    return ret;

flatX = flattenData(X)
flatY = flattenData(Y)  

X=None 
Y=None

trainX,testX,trainY,testY = train_test_split(flatX,flatY,test_size=0.3,random_state = 42,shuffle=True)
flatX=None
flatY=None

In [58]:
from keras import backend as K
smooth = 1

def iou(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(tf.cast(y_true_f,np.float32) * y_pred_f)
    return intersection / (K.sum(tf.cast(y_true_f,np.float32)) + K.sum(y_pred_f) - intersection)

def iou_loss(y_true, y_pred):
    return 1-iou(y_true, y_pred)

In [59]:
modelTrainIgnition =  np.array([x[[9,10,11,12,13]] for x in trainX])
modelTrainSpread =  np.array([x[[30,31,32,33,34,22,23]] for x in trainX])
modelTrainY =  np.array([y[[0]] for y in trainY])
trainX=None
trainY=None

In [75]:
from keras import Input
from keras import Model
import keras



IgnitionInput=Input(shape=(5,30,30))
Ignition= Conv2D(filters=256, kernel_size=(1,1), padding = 'same', activation='relu', input_shape=(7,30,30), data_format='channels_first')(IgnitionInput)
Ignition=Dropout(0.1)(Ignition)
Ignition= Conv2D(filters=512, kernel_size=(1,1), padding = 'same', activation='relu',  data_format='channels_first')(Ignition)
Ignition=Dropout(0.2)(Ignition)
Ignition = Conv2D(filters=1, kernel_size=(1,1), padding = 'same', activation='relu', data_format='channels_first', name="iska_final")(Ignition)
ModelIgnition = Model(inputs=IgnitionInput, output = Ignition)

ModelIgnitionIntermediate = ModelIgnition.get_layer("iska_final").output


SpreadInput= Input(shape=(7,30,30))
Spread =keras.layers.concatenate([ModelIgnitionIntermediate, SpreadInput],axis=1)

Spread= Conv2D(filters=128, kernel_size=(3,3), padding = 'same', activation='relu',  data_format='channels_first')(SpreadInput)
Spread= Conv2D(filters=256, kernel_size=(3,3), padding = 'same', activation='relu', data_format='channels_first')(SpreadInput)
Spread=Dropout(0.1)(Spread)
Spread= Conv2D(filters=512, kernel_size=(3,3), padding = 'same', activation='relu',  data_format='channels_first')(Spread)
Spread=Dropout(0.2)(Spread)
Spread = Conv2D(filters=512, kernel_size=(3,3), padding = 'same', activation='relu', data_format='channels_first')(Spread)
Spread = Dropout(0.2)(Spread)
Spread=Conv2D(filters=512, kernel_size=(3,3), padding = 'same', activation='sigmoid', data_format='channels_first')(Spread)
Spread=Dropout(0.3)(Spread)


X=Conv2D(filters=1, kernel_size=(3,3), padding = 'same', activation='sigmoid', data_format='channels_first')(Spread)


model = Model(inputs= [SpreadInput, ModelIgnitionIntermediate.], outputs=[X])

model.compile(optimizer=Adam(lr = 0.0001), loss=iou_loss, metrics = [iou])

history = model.fit([modelTrainSpread,modelTrainIgnition], modelTrainY, batch_size= 30, epochs=50)

C:\Users\anant\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("is...)`
  del sys.path[0]


AttributeError: 'Tensor' object has no attribute 'inputs'

In [42]:
model1 = Sequential()
output1 = model1.add(Conv2D(filters=128, kernel_size=(3,3), padding = 'same', activation='relu', input_shape=(7,30,30), data_format='channels_first'))
output1 = model1.add(Conv2D(filters=256, kernel_size=(3,3), padding = 'same', activation='relu', input_shape=(22,30,30), data_format='channels_first'))
output1 = model1.add(Dropout(0.1))
output1 = model1.add(Conv2D(filters=512, kernel_size=(3,3), padding = 'same', activation='relu', input_shape=(22,30,30), data_format='channels_first'))
output1 = model1.add(Dropout(0.2))
output1 = model1.add(Conv2D(filters=512, kernel_size=(3,3), padding = 'same', activation='relu', input_shape=(22,30,30), data_format='channels_first'))
output1 = model1.add(Dropout(0.2))
output1 = model1.add(Conv2D(filters=512, kernel_size=(3,3), padding = 'same', activation='sigmoid', input_shape=(22,30,30), data_format='channels_first'))
output1 = model1.add(Dropout(0.3))
output1 = model1.add(Conv2D(filters=1, kernel_size=(3,3), padding = 'same', activation='sigmoid', data_format='channels_first'))



model1.compile(optimizer=Adam(lr = 0.001), loss=iou_loss, metrics = [iou])
history = model1.fit(modelTrainX, modelTrainY, batch_size= 30, epochs=50)

Epoch 1/50
 975/7000 [===>..........................] - ETA: 2:25 - loss: 0.6999 - iou: 0.0013

KeyboardInterrupt: 